# AWS SageMaker & Resources Cleanup Notebook

This notebook provides comprehensive cleanup of AWS resources including:
- SageMaker endpoints, models, and configurations
- Model packages and package groups
- Monitoring schedules
- Feature Store groups
- S3 objects
- Athena tables
- CloudWatch resources (alarms, log groups, dashboards)
- Jupyter notebook kernels

⚠️ **WARNING**: This will delete resources permanently. Review carefully before running.

In [ ]:
import boto3
import time
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Initialize AWS clients
sagemaker_client = boto3.client("sagemaker", region_name="us-east-1")
s3_client = boto3.client("s3")
cloudwatch_client = boto3.client("cloudwatch", region_name="us-east-1")
logs_client = boto3.client("logs", region_name="us-east-1")

try:
    # Get AWS Account ID
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    print(f"Successfully retrieved AWS Account ID: {account_id}")
except Exception as e:
    print(f"Cannot retrieve account information: {e}")
    raise

In [ ]:
print("Starting comprehensive cleanup process...")
print("=" * 80)

## 1. Delete Model Packages

In [ ]:
# Delete Model Packages
print("\nDeleting Model Packages...")
try:
    model_package_group_name = f"venuesignal-model-group-{account_id}"

    list_packages = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        MaxResults=100
    )
    
    if list_packages["ModelPackageSummaryList"]:
        for pkg in list_packages["ModelPackageSummaryList"]:
            pkg_arn = pkg["ModelPackageArn"]
            try:
                sagemaker_client.delete_model_package(ModelPackageName=pkg_arn)
                print(f"   ✓ Model Package deleted: {pkg_arn}")
            except Exception as e:
                print(f"   ✗ Error deleting {pkg_arn}: {str(e)}")
        
        # Wait for deletion to complete
        time.sleep(5)
    else:
        print("   ℹ No model packages found")
except sagemaker_client.exceptions.ResourceNotFound:
    print(f"   ℹ Model Package Group not found")
except Exception as e:
    print(f"   ✗ Error listing model packages: {str(e)}")

## 2. Delete Model Package Group

In [ ]:
# Delete Model Package Group
print("\nDeleting Model Package Group...")
try:
    sagemaker_client.delete_model_package_group(
        ModelPackageGroupName=model_package_group_name
    )
    print(f"   ✓ Model Package Group '{model_package_group_name}' deleted")
except sagemaker_client.exceptions.ResourceNotFound:
    print(f"   ℹ Model Package Group not found (already deleted)")
except Exception as e:
    print(f"   ✗ Error deleting model package group: {str(e)}")

## 3. Delete Monitoring Schedules

In [ ]:
# Delete monitoring schedules
print("\nDeleting monitoring schedules...")
try:
    schedules = sagemaker_client.list_monitoring_schedules()
    if schedules.get('MonitoringScheduleSummaries'):
        for schedule in schedules['MonitoringScheduleSummaries']:
            schedule_name = schedule['MonitoringScheduleName']
            try:
                print(f"   Deleting Schedule: {schedule_name}")
                sagemaker_client.delete_monitoring_schedule(MonitoringScheduleName=schedule_name)
                print(f"   ✓ Schedule '{schedule_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting Schedule: {str(e)}")
            
            time.sleep(5)
    else:
        print("   ℹ No Schedules found")
except Exception as e:
    print(f"   ✗ Error listing Schedule: {str(e)}")

## 4. Delete Endpoints

In [ ]:
# Delete All Endpoints
print("\nDeleting Endpoints...")
try:
    endpoints = sagemaker_client.list_endpoints(MaxResults=100)
    
    if endpoints.get("Endpoints"):
        for endpoint in endpoints["Endpoints"]:
            endpoint_name = endpoint["EndpointName"]
            try:
                print(f"   Deleting Endpoint: {endpoint_name}")
                sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
                print(f"   ✓ Endpoint '{endpoint_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting endpoint: {str(e)}")
        
        # Wait for endpoints to be deleted
        time.sleep(10)
    else:
        print("   ℹ No endpoints found")
except Exception as e:
    print(f"   ✗ Error listing endpoints: {str(e)}")

## 5. Delete Endpoint Configurations

In [ ]:
# Delete Endpoint Configurations
print("\nDeleting Endpoint Configurations...")
try:
    configs = sagemaker_client.list_endpoint_configs(MaxResults=100)
    
    if configs.get("EndpointConfigs"):
        for config in configs["EndpointConfigs"]:
            config_name = config["EndpointConfigName"]
            try:
                print(f"   Deleting Config: {config_name}")
                sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)
                print(f"   ✓ Config '{config_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting config: {str(e)}")
    else:
        print("   ℹ No endpoint configurations found")
except Exception as e:
    print(f"   ✗ Error listing endpoint configurations: {str(e)}")

## 6. Delete Models

In [ ]:
# Delete Models
print("\nDeleting Models...")
try:
    models = sagemaker_client.list_models(MaxResults=100)
    
    if models.get("Models"):
        for model in models["Models"]:
            model_name = model["ModelName"]
            try:
                print(f"   Deleting Model: {model_name}")
                sagemaker_client.delete_model(ModelName=model_name)
                print(f"   ✓ Model '{model_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting model: {str(e)}")
    else:
        print("   ℹ No models found")
except Exception as e:
    print(f"   ✗ Error listing models: {str(e)}")

## 7. Delete Feature Store Groups

In [ ]:
# Delete Feature Store
print("\nDeleting Feature Store Groups...")
try:
    groups = sagemaker_client.list_feature_groups()
    if groups.get("FeatureGroupSummaries"):
        for group in groups["FeatureGroupSummaries"]:
            group_name = group["FeatureGroupName"]
            try:
                print(f"   Deleting feature group: {group_name}")
                !aws sagemaker delete-feature-group --feature-group-name {group_name}
                print(f"   ✓ Feature Group '{group_name}' deleted")
            except Exception as e:
                print(f"   ✗ Error deleting Feature Group: {str(e)}")
    else:
        print("   ℹ No Feature Groups found")
except Exception as e:
    print(f"   ✗ Error listing Feature Groups: {str(e)}")

## 8. Delete S3 Objects

In [ ]:
bucket = f"yelp-aai540-group6-{account_id}"

# Delete S3 Objects
print("\nDeleting S3 Objects...")
try:
    print(f"   Bucket: {bucket}")
    
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket)
    
    delete_count = 0
    for page in pages:
        if 'Contents' in page:
            objects_to_delete = [{'Key': obj['Key']} for obj in page['Contents']]
            if objects_to_delete:
                s3_client.delete_objects(
                    Bucket=bucket,
                    Delete={'Objects': objects_to_delete}
                )
                delete_count += len(objects_to_delete)
                print(f"   Deleted {len(objects_to_delete)} objects...")
    
    print(f"   ✓ Total S3 objects deleted: {delete_count}")
except Exception as e:
    print(f"   ✗ Error deleting S3 objects: {str(e)}")

## 9. Delete Athena Tables

In [ ]:
# Athena Configuration
bucket = f"yelp-aai540-group6-{account_id}"
ATHENA_DB = "yelp"
ATHENA_RESULTS_PREFIX = "/athena/results/"
ATHENA_RESULTS_S3 = f"s3://{bucket}/athena/results/"

print("\nDeleting Athena Tables...")
try:
    # Create PyAthena connection
    conn = connect(
        s3_staging_dir=ATHENA_RESULTS_S3,
        region_name="us-east-1",
        cursor_class=PandasCursor
    )

    print(f"   Connected to Athena database: {ATHENA_DB}")
    print(f"   Results location: {ATHENA_RESULTS_S3}")

    TABLES = ["business", "review", "user", "checkin", "tip", "business_attributes"]

    for table in TABLES:
        query = f"DROP TABLE IF EXISTS {ATHENA_DB}.{table};"
        print(f"   Executing: {query}")
        pd.read_sql(query, conn)

    # Clean up Athena results in S3
    paginator = s3_client.get_paginator("list_objects_v2")
    to_delete = []
    for page in paginator.paginate(Bucket=bucket, Prefix=ATHENA_RESULTS_PREFIX):
        for obj in page.get("Contents", []):
            to_delete.append({"Key": obj["Key"]})

    if not to_delete:
        print(f"   ℹ No results to delete under s3://{bucket}{ATHENA_RESULTS_PREFIX}")
    else:
        # Delete in batches of 1000 (S3 limit)
        for i in range(0, len(to_delete), 1000):
            s3_client.delete_objects(
                Bucket=bucket,
                Delete={"Objects": to_delete[i:i+1000]}
            )
        print(f"   ✓ Deleted {len(to_delete)} Athena result objects")

    print("   ✓ All Athena tables dropped")
except Exception as e:
    print(f"   ✗ Error with Athena cleanup: {str(e)}")

## 10. Delete CloudWatch Resources

This section removes CloudWatch alarms, dashboards, and log groups associated with the ML project.

In [ ]:
# Delete CloudWatch Alarms
print("\nDeleting CloudWatch Alarms...")
try:
    # List all alarms
    paginator = cloudwatch_client.get_paginator('describe_alarms')
    alarm_names = []
    
    for page in paginator.paginate():
        for alarm in page.get('MetricAlarms', []):
            alarm_name = alarm['AlarmName']
            # Filter for SageMaker/project-related alarms (adjust filter as needed)
            if any(keyword in alarm_name.lower() for keyword in ['sagemaker', 'venuesignal', 'endpoint', 'model']):
                alarm_names.append(alarm_name)
    
    if alarm_names:
        # Delete alarms in batches of 100 (API limit)
        for i in range(0, len(alarm_names), 100):
            batch = alarm_names[i:i+100]
            cloudwatch_client.delete_alarms(AlarmNames=batch)
            for alarm_name in batch:
                print(f"   ✓ Alarm deleted: {alarm_name}")
        print(f"   ✓ Total CloudWatch alarms deleted: {len(alarm_names)}")
    else:
        print("   ℹ No CloudWatch alarms found")
except Exception as e:
    print(f"   ✗ Error deleting CloudWatch alarms: {str(e)}")

In [ ]:
# Delete CloudWatch Dashboards
print("\nDeleting CloudWatch Dashboards...")
try:
    # List all dashboards
    dashboards = cloudwatch_client.list_dashboards()
    dashboard_names = []
    
    for dashboard in dashboards.get('DashboardEntries', []):
        dashboard_name = dashboard['DashboardName']
        # Filter for project-related dashboards (adjust filter as needed)
        if any(keyword in dashboard_name.lower() for keyword in ['sagemaker', 'venuesignal', 'ml', 'monitoring']):
            dashboard_names.append(dashboard_name)
    
    if dashboard_names:
        for dashboard_name in dashboard_names:
            try:
                cloudwatch_client.delete_dashboards(DashboardNames=[dashboard_name])
                print(f"   ✓ Dashboard deleted: {dashboard_name}")
            except Exception as e:
                print(f"   ✗ Error deleting dashboard {dashboard_name}: {str(e)}")
        print(f"   ✓ Total CloudWatch dashboards deleted: {len(dashboard_names)}")
    else:
        print("   ℹ No CloudWatch dashboards found")
except Exception as e:
    print(f"   ✗ Error deleting CloudWatch dashboards: {str(e)}")

In [ ]:
# Delete CloudWatch Log Groups
print("\nDeleting CloudWatch Log Groups...")
try:
    # List all log groups
    paginator = logs_client.get_paginator('describe_log_groups')
    log_group_count = 0
    
    for page in paginator.paginate():
        for log_group in page.get('logGroups', []):
            log_group_name = log_group['logGroupName']
            # Filter for SageMaker and project-related log groups
            if any(keyword in log_group_name.lower() for keyword in 
                   ['/aws/sagemaker', 'venuesignal', '/aws/lambda', 'endpoint']):
                try:
                    logs_client.delete_log_group(logGroupName=log_group_name)
                    print(f"   ✓ Log Group deleted: {log_group_name}")
                    log_group_count += 1
                except Exception as e:
                    print(f"   ✗ Error deleting log group {log_group_name}: {str(e)}")
    
    if log_group_count > 0:
        print(f"   ✓ Total CloudWatch log groups deleted: {log_group_count}")
    else:
        print("   ℹ No CloudWatch log groups found")
except Exception as e:
    print(f"   ✗ Error deleting CloudWatch log groups: {str(e)}")

## 11. Shutdown Jupyter Notebook Kernels

This section shuts down all running Jupyter notebook kernels to free up resources.

In [ ]:
# Shutdown all Jupyter notebook kernels
print("\nShutting down Jupyter Notebook Kernels...")
try:
    import requests
    import json
    import os
    
    # Get Jupyter server connection info
    # This works when running in Jupyter/JupyterLab environment
    try:
        from jupyter_server import serverapp
        servers = list(serverapp.list_running_servers())
        
        if not servers:
            print("   ℹ No running Jupyter servers found")
        else:
            for server in servers:
                base_url = server['url']
                token = server.get('token', '')
                
                # Get list of running kernels
                headers = {'Authorization': f'token {token}'} if token else {}
                kernels_url = f"{base_url}api/kernels"
                
                response = requests.get(kernels_url, headers=headers)
                
                if response.status_code == 200:
                    kernels = response.json()
                    
                    if not kernels:
                        print("   ℹ No running kernels found")
                    else:
                        print(f"   Found {len(kernels)} running kernel(s)")
                        
                        for kernel in kernels:
                            kernel_id = kernel['id']
                            kernel_name = kernel.get('name', 'unknown')
                            
                            # Skip the current kernel to avoid stopping this notebook mid-execution
                            # Comment out the next 3 lines if you want to shut down ALL kernels including this one
                            current_kernel = os.environ.get('JPY_PARENT_PID')
                            if kernel_id == current_kernel:
                                print(f"   ⊙ Skipping current kernel: {kernel_id} ({kernel_name})")
                                continue
                            
                            # Shutdown kernel
                            delete_url = f"{base_url}api/kernels/{kernel_id}"
                            delete_response = requests.delete(delete_url, headers=headers)
                            
                            if delete_response.status_code == 204:
                                print(f"   ✓ Kernel shutdown: {kernel_id} ({kernel_name})")
                            else:
                                print(f"   ✗ Error shutting down kernel {kernel_id}: Status {delete_response.status_code}")
                        
                        print("   ✓ Kernel shutdown process completed")
                else:
                    print(f"   ✗ Error getting kernels list: Status {response.status_code}")
    
    except ImportError:
        # Alternative method using IPython kernel manager
        print("   ℹ Trying alternative method...")
        try:
            from IPython import get_ipython
            ipython = get_ipython()
            
            if ipython is not None:
                kernel_manager = ipython.kernel
                print("   ℹ Current kernel will continue running")
                print("   ℹ To shutdown this kernel, use 'Kernel > Shutdown' from the menu")
            else:
                print("   ℹ Not running in IPython environment")
        except Exception as e:
            print(f"   ✗ Alternative method failed: {str(e)}")

except Exception as e:
    print(f"   ✗ Error shutting down kernels: {str(e)}")
    print("   ℹ You may need to manually shutdown kernels from Jupyter interface")

### Alternative: Manual Kernel Shutdown via CLI

If the above method doesn't work in your environment, you can use the following commands to list and shutdown kernels manually.

In [ ]:
# List running Jupyter kernels (CLI method)
!jupyter kernelspec list
print("\nTo see running kernel processes:")
!ps aux | grep -i jupyter | grep -i kernel

## Cleanup Summary

Review the output above to verify all resources have been cleaned up successfully.

In [ ]:
print("\n" + "=" * 80)
print("CLEANUP PROCESS COMPLETED")
print("=" * 80)
print("\nResources cleaned up:")
print("  ✓ Model Packages")
print("  ✓ Model Package Groups")
print("  ✓ Monitoring Schedules")
print("  ✓ Endpoints")
print("  ✓ Endpoint Configurations")
print("  ✓ Models")
print("  ✓ Feature Store Groups")
print("  ✓ S3 Objects")
print("  ✓ Athena Tables")
print("  ✓ CloudWatch Alarms")
print("  ✓ CloudWatch Dashboards")
print("  ✓ CloudWatch Log Groups")
print("  ✓ Jupyter Kernels")
print("\nPlease review the output above for any errors or warnings.")
print("\n⚠️  IMPORTANT REMINDERS:")
print("  - S3 buckets themselves are NOT deleted (only objects inside)")
print("  - IAM roles and policies are NOT deleted")
print("  - VPC and networking resources are NOT deleted")
print("  - Training jobs (completed) remain in history but don't incur costs")
print("\nIf you want to delete these resources, do so manually from AWS Console.")